In [1]:
import sys
import os
import warnings
import torch
import requests
import json
import csv
from esm import Alphabet, FastaBatchedDataset, pretrained

warnings.filterwarnings("ignore")
sys.path.append("./src/")
from CLEAN.infer import *
# pre-compute esm-1b embedding for sequences in new-392 dataset
ensure_dirs("data/esm_data")
ensure_dirs("data/pretrained")

/home/wuke/anaconda3/envs/kcat/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### input 

In [2]:
# input 
taxonomy_id = 'Saccharomyces_cerevisiae'
target_uniprot_fasta_file = 'data/' + taxonomy_id + '_uniprot.fasta'
file_name = taxonomy_id  
target_protein_fasta_file = 'data/' + file_name + '.fasta'
target_protein_csv_file = 'data/'+file_name+'.csv'

# output
target_protein_result_file = './results/'+ file_name +'_maxsep.csv'

### fasta2csv & embedding

In [3]:
def fasta_to_csv(fasta_name, csv_name):
    with open(fasta_name, 'r') as fastafile, open(csv_name, 'w') as outfile:
        csv_rows = []
        fasta_id = ''
        for line in fastafile:
            line = line.strip()
            if line.startswith('>'):
                if fasta_id:
                    csv_rows.append([fasta_id, fasta_sequence])
                fasta_id = line[1:]
                fasta_sequence = ''
            else:
                fasta_sequence += line
        if fasta_id:
            csv_rows.append([fasta_id, fasta_sequence])
        header = ['Entry', 'Sequence']
        csv_rows.insert(0, header)
        csvwriter = csv.writer(outfile, delimiter='\t')
        csvwriter.writerows(csv_rows)

In [4]:
fasta_to_csv(target_protein_fasta_file, target_protein_csv_file)

In [5]:
retrive_esm1b_embedding(file_name)

Transferred model to GPU
Read data/Saccharomyces_cerevisiae_teacher.fasta with 5911 sequences
Processing 1 of 800 batches (77 sequences)
Processing 2 of 800 batches (61 sequences)
Processing 3 of 800 batches (53 sequences)
Processing 4 of 800 batches (47 sequences)
Processing 5 of 800 batches (44 sequences)
Processing 6 of 800 batches (40 sequences)
Processing 7 of 800 batches (38 sequences)
Processing 8 of 800 batches (37 sequences)
Processing 9 of 800 batches (35 sequences)
Processing 10 of 800 batches (34 sequences)
Processing 11 of 800 batches (33 sequences)
Processing 12 of 800 batches (32 sequences)
Processing 13 of 800 batches (31 sequences)
Processing 14 of 800 batches (30 sequences)
Processing 15 of 800 batches (29 sequences)
Processing 16 of 800 batches (29 sequences)
Processing 17 of 800 batches (28 sequences)
Processing 18 of 800 batches (27 sequences)
Processing 19 of 800 batches (27 sequences)
Processing 20 of 800 batches (26 sequences)
Processing 21 of 800 batches (26 se

## predict

In [6]:
from CLEAN.infer import *
# testing on new-392 dataset with pretrained weights trained on 100% split
# infer_pvalue("split100",file_name, p_value=1e-5, nk_random=20,
#              report_metrics=True, pretrained=True)
infer_maxsep("split100",file_name, report_metrics=True, pretrained=True)

The embedding sizes for train and test: torch.Size([241025, 128]) torch.Size([5911, 128])


100%|██████████| 5242/5242 [00:00<00:00, 25334.32it/s]


Calculating eval distance map, between 5911 test ids and 5242 train EC cluster centers


5911it [00:07, 843.80it/s]


############ EC calling results using maximum separation ############
---------------------------------------------------------------------------
>>> total samples: 5911 | total ec: 5848 
>>> precision: 0.0 | recall: 0.0| F1: 0.0 | AUC: 0.5 
---------------------------------------------------------------------------
